In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Ipynb_importer
from utils_x import *
from parameters_x import *

all_col = ['unique_symbol', 'date_time', 'close', 'turnover', 'open', 'high', 'low', 'volume',"use_tag", "adjustment"]
shift_col = ['close', 'turnover', 'open', 'high', 'low', 'volume', 'wap']
p_col = ['close', 'open', 'high', 'low']
v_col = ['turnover', 'volume', 'wap']

print("------------------- load data--------------------")
data = pd.read_csv('index500_v2.csv', dtype=object)
data["adjustment"] = data["adjustment"].fillna(0)

print("------------------- pre data--------------------")
data = data[all_col].copy()
data['turnover'] = data['turnover'].apply(lambda x:np.float(x))
data['volume'] = data['volume'].apply(lambda x:np.float(x))
data['open'] = data['open'].apply(lambda x:np.float(x))
data['close'] = data['close'].apply(lambda x:np.float(x))
data['high'] = data['high'].apply(lambda x:np.float(x))
data['low'] = data['low'].apply(lambda x:np.float(x))
data['adjustment'] = data['adjustment'].apply(lambda x:np.float(x))
print("---------------- pre data step 1-----------------")



importing Jupyter notebook from utils_x.ipynb
importing Jupyter notebook from parameters_x.ipynb
------------------- load data--------------------
------------------- pre data--------------------
---------------- pre data step 1-----------------


In [2]:

data['wap'] = data['turnover']/data['volume']
gb = data.groupby(['unique_symbol'])['open']
data['t+1_open'] = gb.transform(lambda x: x.shift(-1))
data['t+2_open'] = gb.transform(lambda x: x.shift(-2))
print("------------- open shift done --------------------")

------------- open shift done --------------------


In [3]:
gb = data.groupby(['unique_symbol'])['adjustment']
data['t+2_adjustment'] = gb.transform(lambda x: x.shift(-2))
data["y_open"] = data["t+2_open"] - data['t+2_adjustment']
data['return'] = data.apply(lambda x: (x['y_open'])/x['t+1_open']-1, axis=1)
print("------------- y data done --------------------")

------------- y data done --------------------


In [4]:
data = data[~(data['return'] > 0.11)].copy()
data = data[~(data['return'] < -0.11)].copy()

# gb_m = data.groupby(['stock_name'])['money']
# data['buy_money'] = gb_m.transform(lambda x: x.shift(-1))
# data['sell_money'] = gb_m.transform(lambda x: x.shift(-2))
data = data[~data['return'].isnull()].copy()
gb = data.groupby(['date_time'])['return']
data['label'] = gb.transform(lambda x: (x-x.mean())/x.std())
data['rank_label'] = gb.transform(lambda x: pd.qcut(x.rank(method='first'), 10, labels=False))


print("---------------- shift data-------------------")
shift_day = 30
for col in shift_col:
    print(col)
    gb_col = "unique_symbol"
    data = nshift(data, gb_col, col, shift_day)
for col in p_col:
    data = norm_pcol(data, col, shift_day)
for col in v_col:
    data = norm_vcol(data, col, shift_day)

---------------- shift data-------------------
close
turnover
open
high
low
volume
wap


In [5]:
data =  data[data["use_tag"] == "1"].copy()
n_use_f = set(np.array(['unique_symbol', 'date_time', 't+1_open','t+2_open', 'return', 'label', 'rank_label', "use_tag", "adjustment"
                       ,'t+2_adjustment', "y_open"]))
use_f = list(set(np.array(data.columns)) - n_use_f)
print(sorted(use_f))
print(len(use_f))
data.to_csv("index500_v3.csv")

['close', 'close1', 'close10', 'close11', 'close12', 'close13', 'close14', 'close15', 'close16', 'close17', 'close18', 'close19', 'close2', 'close20', 'close21', 'close22', 'close23', 'close24', 'close25', 'close26', 'close27', 'close28', 'close29', 'close3', 'close4', 'close5', 'close6', 'close7', 'close8', 'close9', 'high', 'high1', 'high10', 'high11', 'high12', 'high13', 'high14', 'high15', 'high16', 'high17', 'high18', 'high19', 'high2', 'high20', 'high21', 'high22', 'high23', 'high24', 'high25', 'high26', 'high27', 'high28', 'high29', 'high3', 'high4', 'high5', 'high6', 'high7', 'high8', 'high9', 'low', 'low1', 'low10', 'low11', 'low12', 'low13', 'low14', 'low15', 'low16', 'low17', 'low18', 'low19', 'low2', 'low20', 'low21', 'low22', 'low23', 'low24', 'low25', 'low26', 'low27', 'low28', 'low29', 'low3', 'low4', 'low5', 'low6', 'low7', 'low8', 'low9', 'open', 'open1', 'open10', 'open11', 'open12', 'open13', 'open14', 'open15', 'open16', 'open17', 'open18', 'open19', 'open2', 'open2